In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
# 加载LibSVM格式数据模块
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,roc_curve
from matplotlib import pyplot
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

!pwd
!ls -r ./raw

/home/www/project/jupyter_projects/ReturnToHome
submit_example_A.csv  dataTrain.csv  dataNoLabel.csv  dataA.csv


In [2]:
!pip3 install sklearn

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [2]:
raw_data = pd.read_csv("./raw/dataTrain.csv")

raw_data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,label
0,81167,0,1,mid,0,0,0,153,0,0,...,0,0,0,0,0,0,0,624,1539,0
1,50408,1,1,mid,0,0,21,0,0,0,...,0,0,0,0,0,0,0,186,366,0
2,9114,0,0,high,36,36,120,0,0,0,...,0,0,0,0,0,0,0,24,48,1
3,53228,1,1,low,0,0,0,0,0,0,...,0,0,0,0,0,0,3,3,9,0
4,56280,1,1,mid,9,51,294,0,0,0,...,0,0,0,0,0,0,0,42,141,0


In [11]:
def data_pre_process(raw,is_train=True):
    data = raw.copy()
    data["f3"] = raw["f3"].apply(lambda x:["low","mid","high"].index(x))
    label = None
    
    # 暴力Feature 位置
    loc_f = ['f1', 'f2', 'f4', 'f5', 'f6']
    for i in range(len(loc_f)):
        for j in range(i + 1, len(loc_f)):
            data[f'{loc_f[i]}+{loc_f[j]}'] = data[loc_f[i]] + data[loc_f[j]]
            data[f'{loc_f[i]}-{loc_f[j]}'] = data[loc_f[i]] - data[loc_f[j]]
            data[f'{loc_f[i]}*{loc_f[j]}'] = data[loc_f[i]] * data[loc_f[j]]
            data[f'{loc_f[i]}/{loc_f[j]}'] = data[loc_f[i]] / data[loc_f[j]]
            
    # 暴力Feature 通话
    com_f = ['f43', 'f44', 'f45', 'f46']
    for i in range(len(com_f)):
        for j in range(i + 1, len(com_f)):
            data[f'{com_f[i]}+{com_f[j]}'] = data[com_f[i]] + data[com_f[j]]
            data[f'{com_f[i]}-{com_f[j]}'] = data[com_f[i]] - data[com_f[j]]
            data[f'{com_f[i]}*{com_f[j]}'] = data[com_f[i]] * data[com_f[j]]
            data[f'{com_f[i]}/{com_f[j]}'] = data[com_f[i]] / data[com_f[j]]
    
    data = data.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    if is_train:
        label = data["label"]
        del data["label"]
    del data["id"]
    
    return data,label

data,data_label = data_pre_process(raw_data)
data.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f43*f46,f44+f45,f44-f45,f44*f45,f44+f46,f44-f46,f44*f46,f45+f46,f45-f46,f45*f46
count,59872.000000,59872.000000,59872.000000,59872.000000,59872.000000,59872.000000,59872.000000,59872.000000,59872.000000,59872.000000,...,5.987200e+04,59872.000000,59872.000000,5.987200e+04,59872.000000,59872.000000,5.987200e+04,59872.000000,59872.000000,5.987200e+04
mean,0.306437,0.685746,1.011925,6.003006,18.269024,37.020427,18.458729,6.600130,0.846005,1.249415,...,3.438453e+04,239.177846,-171.512894,3.344295e+04,599.737741,-532.072789,9.086314e+04,771.250635,-360.559894,4.747821e+05
std,0.461017,0.464222,0.818004,14.961573,32.672944,56.295802,46.338323,30.300542,9.605836,11.978737,...,5.783971e+05,501.240282,381.126786,5.639386e+05,1156.715686,1025.240331,1.495052e+06,1435.202243,790.871166,4.415698e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,-12069.000000,0.000000e+00,0.000000,-28518.000000,0.000000e+00,0.000000,-19824.000000,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,15.000000,-201.000000,0.000000e+00,42.000000,-618.000000,0.000000e+00,63.000000,-429.000000,2.700000e+02
50%,0.000000,1.000000,1.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,93.000000,-66.000000,0.000000e+00,255.000000,-225.000000,0.000000e+00,348.000000,-144.000000,1.696500e+04
75%,1.000000,1.000000,2.000000,0.000000,27.000000,57.000000,0.000000,0.000000,0.000000,0.000000,...,1.748250e+03,267.000000,-6.000000,2.430000e+03,690.000000,-30.000000,7.029000e+03,900.000000,-15.000000,1.318680e+05
max,1.000000,1.000000,2.000000,267.000000,744.000000,1365.000000,273.000000,264.000000,162.000000,237.000000,...,7.881943e+07,18915.000000,17793.000000,8.005315e+07,32307.000000,17394.000000,2.155913e+08,42015.000000,11688.000000,3.821713e+08


In [12]:
x_train,x_test,y_train,y_test = train_test_split(data, data_label, test_size = 0.2,random_state = 33)
x_train

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f43*f46,f44+f45,f44-f45,f44*f45,f44+f46,f44-f46,f44*f46,f45+f46,f45-f46,f45*f46
24464,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,12,-12,0,12,-12,0
32995,0,1,1,0,15,33,0,0,0,0,...,0,9,-9,0,39,-39,0,48,-30,351
35422,0,1,0,0,0,9,0,0,0,0,...,0,171,-147,1908,453,-429,5292,600,-282,70119
46630,1,1,0,0,0,0,0,0,0,0,...,0,9,-9,0,21,-21,0,30,-12,189
36473,0,1,2,21,54,162,0,0,0,0,...,18090,357,-207,21150,678,-528,45225,885,-321,170046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27282,0,1,1,15,15,15,42,0,0,0,...,387090,1383,-561,399492,2757,-1935,964206,3318,-1374,2280312
578,0,0,0,0,0,0,0,0,0,0,...,11070,174,-6,7560,453,-285,30996,459,-279,33210
38616,0,0,2,24,24,24,0,0,0,0,...,2304,222,-210,1296,390,-378,2304,600,-168,82944
2439,1,1,1,0,0,0,0,0,0,54,...,0,15,-15,0,21,-21,0,36,-6,315


In [13]:
### 训练模型
model = xgb.XGBClassifier(learning_rate=0.1,
                      n_estimators=100,            # 树的个数--100棵树建立xgboost
                      max_depth=6,                 # 树的深度
                      min_child_weight = 1,        # 叶子节点最小权重
                      gamma=0.,                    # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,               # 随机选择80%样本建立决策树
                      colsample_btree=0.8,         # 随机选择80%特征建立决策树
                      objective='binary:logistic', # 指定损失函数
                      scale_pos_weight=1,          # 解决样本个数不平衡的问题
                      random_state=27              # 随机数
                      )

# 拟合
model.fit(x_train, y_train, eval_set = [(x_test,y_test)], eval_metric = "auc", early_stopping_rounds = 10,verbose = True)

/usr/local/lib64/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:22:45] WARNING: ../src/learner.cc:576: 
Parameters: { "colsample_btree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.85191
[1]	validation_0-auc:0.85630
[2]	validation_0-auc:0.85676
[3]	validation_0-auc:0.85688
[4]	validation_0-auc:0.85755
[5]	validation_0-auc:0.85820
[6]	validation_0-auc:0.85860
[7]	validation_0-auc:0.85860
[8]	validation_0-auc:0.85962
[9]	validation_0-auc:0.86022
[10]	validation_0-auc:0.86095
[11]	validation_0-auc:0.86128
[12]	validation_0-auc:0.86163
[13]	validation_0-auc:0.86169
[14]	validation_0-auc:0.86203
[15]	validation_0-auc:0.86245
[16]	validation_0-auc:0.86275
[17]	validation_0-auc:0.86285
[18]	validation_0-auc:0.86312
[19]	validation_0-auc:0.86329
[20]	validation_0-auc:0.86334
[21]	validatio

XGBClassifier(base_score=0.5, booster='gbtree', colsample_btree=0.8,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              enable_categorical=False, gamma=0.0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=1, num_parallel_tree=1, predictor='auto',
              random_state=27, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [15]:
# 查看训练模型准确度
def show_auc(x,y,model):
    pred_label = model.predict(x)
    pred_prob = model.predict_proba(x)
    acc = accuracy_score(y, pred_label)
    precision = precision_score(y, pred_label)
    recall = recall_score(y, pred_label)
    auc = roc_auc_score(y, pred_label) # 标签计算
    fpr,tpr,threshold=roc_curve(y,pred_prob[:,1],pos_label=1)
    auc_all = roc_auc_score(y, pred_prob[:,1]) # 按概率计算
    print("===="*8)
    print("Summary:")
    print(f"acc: {acc}\nrecall: {recall}\nauc: {auc}\nfull_auc: {auc_all}")
    print("===="*8)
    print()

show_auc(x_train, y_train, model)
show_auc(x_test, y_test, model)
show_auc(data,data_label,model)

Summary:
acc: 0.8601791343925507
recall: 0.5288469636991493
auc: 0.7491142435639873
full_auc: 0.8970883035124922

Summary:
acc: 0.8443423799582463
recall: 0.4913086257789439
auc: 0.7281212633712106
full_auc: 0.8659055723430441

Summary:
acc: 0.8570116247995724
recall: 0.5211767859536255
auc: 0.7448375587165235
full_auc: 0.8908032417439801



In [ ]:
# 半监督
raw_semi

In [17]:
# 测试提交
raw_A = pd.read_csv("./raw/dataA.csv")
data_A,_ = data_pre_process(raw_A,is_train=False)

print(data_A.head())

pred_label_A = model.predict(data_A)
pred_label_A

   f1  f2  f3  f4  f5  f6  f7  f8  f9  f10  ...  f43*f46  f44+f45  f44-f45  \
0   1   1   1   0   0   0   0   0   0    0  ...        0       57      -45   
1   0   1   1   0   0   0   0   0   0    0  ...     2754      240     -198   
2   0   1   2   0   0   0   0   0   0    0  ...        0       18      -18   
3   0   1   1   0   0   0   0   0   0    0  ...        0        9       -9   
4   1   1   0   0   0   0   0   0   0    0  ...        0       78      -78   

   f44*f45  f44+f46  f44-f46  f44*f46  f45+f46  f45-f46  f45*f46  
0      306      174     -162     1008      219     -117     8568  
1     4599      480     -438     9639      678     -240   100521  
2        0       27      -27        0       45       -9      486  
3        0        9       -9        0       18        0       81  
4        0      243     -243        0      321     -165    18954  

[5 rows x 94 columns]


array([0, 0, 0, ..., 0, 1, 0])

In [19]:
def save_label(data_id,label):
    df = pd.DataFrame({'id':data_id,'label':label})
    df.to_csv("./res/s.csv",index=False)

save_label(list(raw_A['id']),list(pred_label_A))

In [24]:
!echo "# CCF-BDCI-2022-FXFZ" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/HoBeedzc/CCF-BDCI-2022-FXFZ.git
!git push -u origin main

初始化空的 Git 版本库于 /home/www/project/jupyter_projects/ReturnToHome/.git/
[master（根提交） 387103b] first commit
 1 file changed, 1 insertion(+)
 create mode 100644 README.md
Username for 'https://github.com': ^C


In [25]:
!pwd

/home/www/project/jupyter_projects/ReturnToHome
